# _dim_product table_

Create the Table

In [0]:
%sql
CREATE TABLE IF NOT EXISTS bikes.gold.dim_product (
  product_id INT,
  product_key STRING,
  product_name STRING,
  product_cost INT,
  product_line STRING,
  category_name STRING,
  subcategory_name STRING,
  maintenance_required BOOLEAN,
  start_date DATE,
  end_date DATE
)
COMMENT "Product dimension table containing products details"

Addin the commets to the columns and tags

In [0]:
%sql
ALTER TABLE bikes.gold.dim_product ALTER COLUMN product_id COMMENT 'Surrogate product identifier used in the Gold layer';
ALTER TABLE bikes.gold.dim_product ALTER COLUMN product_key COMMENT 'Business product key used consistently across all source systems';
ALTER TABLE bikes.gold.dim_product ALTER COLUMN product_name COMMENT 'Product name as provided by the source systems';
ALTER TABLE bikes.gold.dim_product ALTER COLUMN product_cost COMMENT 'Standard cost of the product used for reporting and analytics';
ALTER TABLE bikes.gold.dim_product ALTER COLUMN product_line COMMENT 'Product line or product family classification';
ALTER TABLE bikes.gold.dim_product ALTER COLUMN category_name COMMENT 'High-level product category for analytical grouping';
ALTER TABLE bikes.gold.dim_product ALTER COLUMN subcategory_name COMMENT 'Detailed product subcategory within the main category';
ALTER TABLE bikes.gold.dim_product ALTER COLUMN maintenance_required COMMENT 'Flag indicating whether the product requires maintenance';
ALTER TABLE bikes.gold.dim_product ALTER COLUMN start_date COMMENT 'Date when the product became available or active';
ALTER TABLE bikes.gold.dim_product ALTER COLUMN end_date COMMENT 'Date when the product was discontinued or became inactive';

Inserting the data by aggregation product and category tables form CRM and ERP

In [0]:
%sql
INSERT OVERWRITE TABLE bikes.gold.dim_product
SELECT 
    p.product_id,
    p.product_key,
    p.product_name,
    p.product_cost,
    p.product_line,
    c.category_name,
    c.subcategory_name,
    c.maintenance_required,
    p.start_date,
    p.end_date
FROM bikes.silver.crm_sales_details s
LEFT JOIN bikes.silver.crm_product_info p
    ON p.product_key LIKE concat('%', s.sls_prd_key)
LEFT JOIN bikes.silver.erp_category c
    ON regexp_replace(
         substring(p.product_key, 1, 5),
         '-', '_'
       ) = c.product_category_key

Add tags to table

In [0]:
%sql
ALTER TABLE bikes.gold.dim_product
SET TAGS (
  'layer' = 'gold',
  'table_type' = 'dimension',
  'domain' = 'product',
  'pii' = 'false'
);
